In [37]:
# importing required libraries
import os
import subprocess
import stat
import re
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

In [2]:
# Replace 'your_file.csv' with the actual path to your CSV file
file_path = r"C:\Users\RATNADEEP\autos.csv"

# Try reading the CSV file with a different encoding
try:
    df = pd.read_csv(file_path, encoding='utf-8')
except UnicodeDecodeError:
    # If reading with utf-8 fails, try other encodings
    try:
        df = pd.read_csv(file_path, encoding='ISO-8859-1')
    except UnicodeDecodeError:
        # If 'ISO-8859-1' also fails, try 'utf-16'
        df = pd.read_csv(file_path, encoding='utf-16')

# Continue with the rest of your data processing using the 'df' DataFrame

In [3]:
df #Raw Data

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-24 11:52:17,Golf_3_1.6,privat,Angebot,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,A5_Sportback_2.7_Tdi,privat,Angebot,18300,test,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,"Jeep_Grand_Cherokee_""Overland""",privat,Angebot,9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,GOLF_4_1_4__3TÜRER,privat,Angebot,1500,test,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,Skoda_Fabia_1.4_TDI_PD_Classic,privat,Angebot,3600,test,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371523,2016-03-14 17:48:27,Suche_t4___vito_ab_6_sitze,privat,Angebot,2200,test,NaN,2005,NaN,0,NaN,20000,1,NaN,sonstige_autos,NaN,2016-03-14 00:00:00,0,39576,2016-04-06 00:46:52
371524,2016-03-05 19:56:21,Smart_smart_leistungssteigerung_100ps,privat,Angebot,1199,test,cabrio,2000,automatik,101,fortwo,125000,3,benzin,smart,nein,2016-03-05 00:00:00,0,26135,2016-03-11 18:17:12
371525,2016-03-19 18:57:12,Volkswagen_Multivan_T4_TDI_7DC_UY2,privat,Angebot,9200,test,bus,1996,manuell,102,transporter,150000,3,diesel,volkswagen,nein,2016-03-19 00:00:00,0,87439,2016-04-07 07:15:26
371526,2016-03-20 19:41:08,VW_Golf_Kombi_1_9l_TDI,privat,Angebot,3400,test,kombi,2002,manuell,100,golf,150000,6,diesel,volkswagen,NaN,2016-03-20 00:00:00,0,40764,2016-03-24 12:45:21


In [4]:
df.isnull().sum()

dateCrawled                0
name                       0
seller                     0
offerType                  0
price                      0
abtest                     0
vehicleType            37869
yearOfRegistration         0
gearbox                20209
powerPS                    0
model                  20484
kilometer                  0
monthOfRegistration        0
fuelType               33386
brand                      0
notRepairedDamage      72060
dateCreated                0
nrOfPictures               0
postalCode                 0
lastSeen                   0
dtype: int64

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 371528 entries, 0 to 371527
Data columns (total 20 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   dateCrawled          371528 non-null  object
 1   name                 371528 non-null  object
 2   seller               371528 non-null  object
 3   offerType            371528 non-null  object
 4   price                371528 non-null  int64 
 5   abtest               371528 non-null  object
 6   vehicleType          333659 non-null  object
 7   yearOfRegistration   371528 non-null  int64 
 8   gearbox              351319 non-null  object
 9   powerPS              371528 non-null  int64 
 10  model                351044 non-null  object
 11  kilometer            371528 non-null  int64 
 12  monthOfRegistration  371528 non-null  int64 
 13  fuelType             338142 non-null  object
 14  brand                371528 non-null  object
 15  notRepairedDamage    299468 non-nu

In [96]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
price,371524.0,17295.302511,3.587973e+06,0.0,1150.0,2950.0,7200.0,2.147484e+09
yearOfRegistration,371524.0,2004.578033,9.286710e+01,1000.0,1999.0,2003.0,2008.0,9.999000e+03
kilometer,371524.0,125618.842928,4.011214e+04,5000.0,125000.0,150000.0,150000.0,1.500000e+05
powerPS,371524.0,115.549039,1.921405e+02,0.0,70.0,105.0,150.0,2.000000e+04
postalCode,371524.0,50820.494270,2.579909e+04,1067.0,30459.0,49610.0,71546.0,9.999800e+04


## Changing str to datetime

In [7]:
df['dateCreated']=pd.to_datetime(df['dateCreated']) #Changing str to datetime

In [8]:
df['dateCrawled']=pd.to_datetime(df['dateCrawled']) #Changing str to datetime

In [10]:
df['lastSeen']=pd.to_datetime(df['lastSeen']) #Changing str to datetime

## Translating German to English

In [11]:
# checking the different types of values in the column seller
df.seller.unique()

array(['privat', 'gewerblich'], dtype=object)

In [12]:
#changing values of seller  privat to private and  gewerblich to commercial
df['seller'].replace({"privat" : "private", "gewerblich" : "commercial"}, inplace=True)

In [13]:
df.seller.unique()

array(['private', 'commercial'], dtype=object)

In [14]:
# checking the different types of values in the column offerType
df.offerType.unique()

array(['Angebot', 'Gesuch'], dtype=object)

In [15]:
# changing values of offerType Gesuch to Request and Angebot to Offer
df['offerType'].replace({'Angebot':'Offer', 'Gesuch': 'Request'}, inplace = True)

In [16]:
df.offerType.unique()

array(['Offer', 'Request'], dtype=object)

In [18]:
# checking the different types of values in the column gearbox
df.gearbox.unique()

array(['manuell', 'automatik', nan], dtype=object)

In [19]:
# changing values of gearbox to manuell to manual and automatik to automatic
df['gearbox'].replace({'manuell': 'manual', 'automatik': 'automatic'}, inplace =True)

In [20]:
df.gearbox.unique()

array(['manual', 'automatic', nan], dtype=object)

In [22]:
# checking the different types of values in the column fueltype
df.fuelType.unique()

array(['benzin', 'diesel', nan, 'lpg', 'andere', 'hybrid', 'cng',
       'elektro'], dtype=object)

In [23]:
# changing values of fueltype to benzin to petrol, andere to  other,and elektro to electric
df['fuelType'].replace({'benzin': 'petrol', 'andere': 'other','elektro': 'electric'}, inplace =True)

In [24]:
df.fuelType.unique()

array(['petrol', 'diesel', nan, 'lpg', 'other', 'hybrid', 'cng',
       'electric'], dtype=object)

In [25]:
# checking the different types of values in the column notRepairedDamage
df.notRepairedDamage.unique()  

array([nan, 'ja', 'nein'], dtype=object)

In [26]:
# changing values of notRepairedDamage to  ja to yes and nein  to no
df['notRepairedDamage'].replace({'ja': 'yes', 'nein': 'no' } ,inplace =True)

In [31]:
df.notRepairedDamage.unique()  

array([nan, 'yes', 'no'], dtype=object)

## Removing Null Values

In [43]:
# checking the column vehicleType for null values 
df["vehicleType"].isnull().values.sum()

37869

In [44]:
# changing the vehicleType from NaN to Others
df["vehicleType"].fillna("Other", inplace=True)

In [45]:
# checking if there are any null values in the column brand
df["brand"].isnull().value_counts()

False    371528
Name: brand, dtype: int64

In [46]:
# checking if there are any null values in the column seller
df["seller"].isnull().value_counts()

False    371528
Name: seller, dtype: int64

In [47]:
# checking if there are any null values in the offerType 
df["offerType"].isnull().value_counts()

False    371528
Name: offerType, dtype: int64

In [48]:
# checking if there are any null values in the yearOfRegistration
df["yearOfRegistration"].isnull().value_counts()

False    371528
Name: yearOfRegistration, dtype: int64

In [49]:
# checking if the gearbox column has null values
df["gearbox"].isnull().value_counts()


False    351319
True      20209
Name: gearbox, dtype: int64

In [50]:
# setting the NaN gearbox types to Unspecified
df["gearbox"].fillna("Unspecified", inplace=True)

In [51]:
# checking if the fuelType has null values
df["fuelType"].isnull().value_counts()

False    338142
True      33386
Name: fuelType, dtype: int64

In [52]:
# setting the NaN fuelType types to other
df["fuelType"].fillna("other",inplace=True)

In [53]:
# deleting the column noOfPictures since all of them are Zero
del df["nrOfPictures"]

In [54]:
# checking for null values in powerPS column
df["powerPS"].isnull().value_counts()

False    371528
Name: powerPS, dtype: int64

In [55]:
# checking for unique values in notRepairedDamage
df["notRepairedDamage"].isnull().value_counts()

False    299468
True      72060
Name: notRepairedDamage, dtype: int64

In [56]:
# setting nan in notRepairedDamage to other
df["notRepairedDamage"].fillna("other",inplace=True)

In [57]:
# checking for unique values in model column
df["model"].isnull().value_counts()

False    351044
True      20484
Name: model, dtype: int64

In [58]:
# setting nan in model column to Other
df["model"].fillna("Other",inplace=True)

In [99]:
# checking for null values in abtest column
df["abtest"].isnull().value_counts()

False    371524
Name: abtest, dtype: int64

In [112]:
df['monthOfRegistration'].replace(0,1,inplace=True) ## Replacing 0 with 1 since 0 doesn't refer to any month.

In [113]:
df['monthOfRegistration'].replace({1:'January',2:'February',3:'March',4:'April',5:'May',6:'June',7:'July',8:'August',9:'September',10:'October',11:'November',12:'December'},inplace=True)

In [101]:
# checking if postal code values are null
df["postalCode"].isnull().value_counts()

False    371524
Name: postalCode, dtype: int64

In [63]:
df.isnull().sum()

dateCrawled            0
name                   0
seller                 0
offerType              0
price                  0
abtest                 0
vehicleType            0
yearOfRegistration     0
gearbox                0
powerPS                0
model                  0
kilometer              0
monthOfRegistration    0
fuelType               0
brand                  0
notRepairedDamage      0
dateCreated            0
postalCode             0
lastSeen               0
dtype: int64

## Removing underscores

In [64]:
def substitution(x):
    return re.sub(r'[_]','',x)

In [65]:
df['name']=df['name'].apply(substitution)    #Removing '_' in name column.

In [66]:
df['name'].unique()

array(['Golf31.6', 'A5Sportback2.7Tdi', 'JeepGrandCherokee"Overland"',
       ..., 'Smartsmartleistungssteigerung100ps', 'VWGolfKombi19lTDI',
       'BMWM135ivollausgestattetNP52.720Euro'], dtype=object)

In [67]:
df['brand']=df['brand'].apply(substitution)  #Removing '_' in name column.

In [68]:
df['brand'].unique()

array(['volkswagen', 'audi', 'jeep', 'skoda', 'bmw', 'peugeot', 'ford',
       'mazda', 'nissan', 'renault', 'mercedesbenz', 'opel', 'seat',
       'citroen', 'honda', 'fiat', 'mini', 'smart', 'hyundai',
       'sonstigeautos', 'alfaromeo', 'subaru', 'volvo', 'mitsubishi',
       'kia', 'suzuki', 'lancia', 'porsche', 'toyota', 'chevrolet',
       'dacia', 'daihatsu', 'trabant', 'saab', 'chrysler', 'jaguar',
       'daewoo', 'rover', 'landrover', 'lada'], dtype=object)

In [69]:
def sub_model(x):
    return re.sub(r'[_]+?','',x)

In [73]:
df['model']=df['model'].apply(sub_model) ## Removing '_' from model column.

## Removing Duplicates

In [71]:
df[df.duplicated()]

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,postalCode,lastSeen
179448,2016-03-08 18:42:48,MercedesBenzCLKCoupe230KompressorSport,private,Offer,1799,test,coupe,1999,automatic,193,clk,20000,Jul,petrol,mercedesbenz,no,2016-03-08,89518,2016-03-09 09:46:57
242556,2016-03-28 00:56:10,SuzukiIgnis,private,Offer,1000,control,kleinwagen,2002,manual,83,andere,150000,Jan,petrol,suzuki,no,2016-03-28,66589,2016-03-28 08:46:21
270719,2016-04-03 09:01:15,MercedesBenzCLK320W209,private,Offer,4699,test,coupe,2003,automatic,218,clk,125000,Jun,petrol,mercedesbenz,yes,2016-04-03,75196,2016-04-07 09:44:54
341474,2016-03-18 18:46:15,VolkswagenPassatVariant1.9TDIHighline,private,Offer,1999,control,kombi,2001,manual,131,passat,150000,Jul,diesel,volkswagen,no,2016-03-18,36391,2016-03-18 18:46:15


In [74]:
df.duplicated().sum()

4

In [75]:
df.drop_duplicates(inplace=True)

In [76]:
df.duplicated().sum()

0

## Changing the column order

In [77]:
column_order=['name','seller','model','brand','offerType','price','yearOfRegistration','monthOfRegistration','vehicleType','gearbox','fuelType','kilometer','notRepairedDamage','abtest','powerPS','postalCode','dateCrawled','dateCreated','lastSeen']

In [78]:
df=df[column_order]

In [79]:
df.head()

,name,seller,model,brand,offerType,price,yearOfRegistration,monthOfRegistration,vehicleType,gearbox,fuelType,kilometer,notRepairedDamage,abtest,powerPS,postalCode,dateCrawled,dateCreated,lastSeen
0,Golf31.6,private,golf,volkswagen,Offer,480,1993,Jan,Other,manual,petrol,150000,other,test,0,70435,2016-03-24 11:52:17,2016-03-24,2016-04-07 03:16:57
1,A5Sportback2.7Tdi,private,Other,audi,Offer,18300,2011,May,coupe,manual,diesel,125000,yes,test,190,66954,2016-03-24 10:58:45,2016-03-24,2016-04-07 01:46:50
2,"JeepGrandCherokee""Overland""",private,grand,jeep,Offer,9800,2004,Aug,suv,automatic,diesel,125000,other,test,163,90480,2016-03-14 12:52:21,2016-03-14,2016-04-05 12:47:46
3,GOLF4143TÜRER,private,golf,volkswagen,Offer,1500,2001,Jun,kleinwagen,manual,petrol,150000,no,test,75,91074,2016-03-17 16:54:04,2016-03-17,2016-03-17 17:40:17
4,SkodaFabia1.4TDIPDClassic,private,fabia,skoda,Offer,3600,2008,Jul,kleinwagen,manual,diesel,90000,no,test,69,60437,2016-03-31 17:25:20,2016-03-31,2016-04-06 10:17:21


## Saving the dataframe in a pickle file

In [117]:
import pickle as pkl

In [118]:
with open('german_df.pkl','wb') as f:
    pkl.dump(df,f)

In [119]:
with open("german_df.pkl","rb") as f:
    ebay_cars = pkl.load(f)

In [120]:
df=ebay_cars.copy()

In [121]:
df

,name,seller,model,brand,offerType,price,yearOfRegistration,monthOfRegistration,vehicleType,gearbox,fuelType,kilometer,notRepairedDamage,abtest,powerPS,postalCode,dateCrawled,dateCreated,lastSeen
0,Golf31.6,private,golf,volkswagen,Offer,480,1993,Jan,Other,manual,petrol,150000,other,test,0,70435,2016-03-24 11:52:17,2016-03-24,2016-04-07 03:16:57
1,A5Sportback2.7Tdi,private,Other,audi,Offer,18300,2011,May,coupe,manual,diesel,125000,yes,test,190,66954,2016-03-24 10:58:45,2016-03-24,2016-04-07 01:46:50
2,"JeepGrandCherokee""Overland""",private,grand,jeep,Offer,9800,2004,Aug,suv,automatic,diesel,125000,other,test,163,90480,2016-03-14 12:52:21,2016-03-14,2016-04-05 12:47:46
3,GOLF4143TÜRER,private,golf,volkswagen,Offer,1500,2001,Jun,kleinwagen,manual,petrol,150000,no,test,75,91074,2016-03-17 16:54:04,2016-03-17,2016-03-17 17:40:17
4,SkodaFabia1.4TDIPDClassic,private,fabia,skoda,Offer,3600,2008,Jul,kleinwagen,manual,diesel,90000,no,test,69,60437,2016-03-31 17:25:20,2016-03-31,2016-04-06 10:17:21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371523,Suchet4vitoab6sitze,private,Other,sonstigeautos,Offer,2200,2005,Jan,Other,Unspecified,other,20000,other,test,0,39576,2016-03-14 17:48:27,2016-03-14,2016-04-06 00:46:52
371524,Smartsmartleistungssteigerung100ps,private,fortwo,smart,Offer,1199,2000,Mar,cabrio,automatic,petrol,125000,no,test,101,26135,2016-03-05 19:56:21,2016-03-05,2016-03-11 18:17:12
371525,VolkswagenMultivanT4TDI7DCUY2,private,transporter,volkswagen,Offer,9200,1996,Mar,bus,manual,diesel,150000,no,test,102,87439,2016-03-19 18:57:12,2016-03-19,2016-04-07 07:15:26
371526,VWGolfKombi19lTDI,private,golf,volkswagen,Offer,3400,2002,Jun,kombi,manual,diesel,150000,other,test,100,40764,2016-03-20 19:41:08,2016-03-20,2016-03-24 12:45:21
